In [ ]:
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from PIL import Image as PILImage, PngImagePlugin
import io

In [ ]:

import rclpy
import rclpy.logging
from rclpy.executors import MultiThreadedExecutor
import cv2
import torch
import time
import numpy as np

rclpy.init(args=None)
image_subscriber = ImageSubscriber()

from utils import *
rscalc = RealSensePointCalculator()

executor = MultiThreadedExecutor()
executor.add_node(image_subscriber)
executor.add_node(rscalc)


In [ ]:
try:
    while rclpy.ok():
        print(1)
        executor.spin_once(timeout_sec=0.1)
        if image_subscriber.pil_image is not None:
            print(2)
            pil_image = image_subscriber.pil_image
            if rscalc.info_received():
                break
        time.sleep(0.1)
        print(3)

except KeyboardInterrupt:
    pass

finally:
    pass

In [ ]:
ct = rscalc.get_clock().now().nanoseconds * 1e-9
print(ct)


In [ ]:
print(pil_image.size)
# display the image using matplotlib
import matplotlib.pyplot as plt
plt.imshow(pil_image)
# plt.imshow(image[0].permute(1, 2, 0).cpu())

In [ ]:
# Before running this, add lang-seg folder to your python path
from LoadLSeg import *
import torchvision.transforms as transforms
# from PIL import Image # use PILImage instead
from test_lseg_zs import *

transform = transforms.Compose([
    transforms.Resize(480),           # Resize the shorter side to 480 while maintaining aspect ratio
    transforms.CenterCrop((480, 480)),  # Crop the center to 480x480
    transforms.ToTensor()            # Convert to tensor
])

image = PILImage.open("/home/fyp/Pictures/TestSeg/bottle_keyboard.jpg")
image_tensor = transform(image)
image_tensor.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_tensor.permute(1, 2, 0))

In [ ]:
torch.cuda.empty_cache()
with torch.no_grad():
    feat = model(image_tensor.unsqueeze(0).cuda())
feat.shape

In [ ]:
search = "keybaord"
threshold = 0.96
text_feat1 = model.encode_text(search)
sim_image = feat.half() @ text_feat1.t()
sim_image[sim_image < sim_image.max()*threshold] = 0
display_img2 = sim_image.detach().cpu().numpy()
plt.imshow(display_img2, cmap='gray', vmin=display_img2.min(), vmax=display_img2.max())
# coordinates of the maximum value in the similarity image
print("###############################################")
print(f"searching for: {search}")
print("###############################################")
x, y = float((sim_image.argmax() // sim_image.shape[1]).detach().cpu()), float((sim_image.argmax() % sim_image.shape[1]).detach().cpu())
print("max value at (coordinate in pixel):", f"row {x}", f"col {y}")
print("max value at (coordinate %):", round(x/480,2), round(y/480, 2))

In [ ]:
os.chdir("/home/fyp/llmbot2_ws/src/sem_map/sem_map")
from map_utils import PCA_and_Cluster

In [ ]:
clustered_image = PCA_and_Cluster(feat)
plt.imshow(clustered_image, cmap='gray', vmin=clustered_image.min(), vmax=clustered_image.max())

In [ ]:
key_pixels = obtain_key_pixels(feat, clustered_image)
key_points = rscalc.obtain_key_points(key_pixels)

In [ ]:
import tf2_ros
camera_pose = tf2_ros.TransformStamped()
type(camera_pose.transform.rotation)

In [ ]:
import tf_transformations
point = np.array([0.5, 0.5, 1])
translation = np.array([camera_pose.transform.translation.x,
                        camera_pose.transform.translation.y,
                        camera_pose.transform.translation.z])
rotation = [camera_pose.transform.rotation.x,
            camera_pose.transform.rotation.y,
            camera_pose.transform.rotation.z,
            camera_pose.transform.rotation.w]
rotation_matrix = tf_transformations.quaternion_matrix(rotation)[:3, :3]
transformed_point = np.dot(rotation_matrix, point) + translation
print(point)

In [1]:
import numpy as np
points = [np.array([0.5, 0.5, 1]), np.array([0.5, 0.5, 1]), np.array([0.5, 0.5, 1]), np.array([0.5, 0.5, 1]), np.array([0.5, 0.5, 1]), np.array([0.5, 0.5, 1])]
key_i = np.arange(int(len(points)*0.5))
key_i

array([0, 1, 2])

In [ ]:

executor.shutdown()
image_subscriber.destroy_node()
rscalc.destroy_node()
rclpy.shutdown()